In [17]:
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None,*,value = None):
        self.feature = feature #which feature was devived with
        self.threshold = threshold #threshold value
        self.left = left #left child
        self.right = right #right child
        self.value = value #value of the node

        def is_leaf_node(self):
            return self.value is not None


In [22]:
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from collections import Counter

class DecisionTree:

    def  __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=5, random_state=0, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state #seed
        self.criterion = criterion
        self.root = None   # root of the tree 

    def _most_common_label(self, y):
        #leggyakoribb label meghatarozasa
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def _entropy(self, y): 
        #entropy meghatarozasa
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])
    
    def _split(self, X_column, split_treshhold):
        #adatok elvalasztasa
        left_mask = np.argwhere(X_column <= split_treshhold).flatten()
        right_mask = np.argwhere(X_column > split_treshhold).flatten()
        return left_mask, right_mask

    def _informationn_gain(self, y, X_column, split_treshhold):
        #information gain meghatarozasa

        #parent entropy
        parent_entropy = self._entropy(y)

        left_mask, right_mask = self._split(X_column, split_treshhold)

        if (len(left_mask) == 0 or len(right_mask) == 0):
            return 0
        """left_mask = X_column <= split_treshhold
        right_mask = X_column > split_treshhold

        if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
            return 0

        if len(y[left_mask]) > 0:
            left_entropy = self._entropy(y[left_mask])
        else:
            left_entropy = 0

        if len(y[right_mask]) > 0:
            right_entropy = self._entropy(y[right_mask])
        else:
            right_entropy = 0"""
        #child entropy
        #child_entropy = len(y[left_mask])/len(y)*left_entropy + len(y[right_mask])/len(y)*right_entropy

        n = len(y)
        n_l, n_r = len(left_mask), len(right_mask)
        e_l, e_r = self._entropy(y[left_mask]), self._entropy(y[right_mask])

        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r


        #calculate information gain
        ig = parent_entropy - child_entropy

        return ig
    
    def _best_split(self, X, y, feat_idxs):
        #legjobb elvalasztas meghatarozasa
        best_gain = -1
        split_idx, split_treshhold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:,feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                #calculate information gain
                print("feat_idx: ", feat_idx, "threshold: ", threshold)
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_treshhold = threshold

        return split_idx, split_treshhold

    def _grow_tree(self, X, y, depth=0):
        number_of_samples, number_of_features = X.shape
        number_of_labels = len(np.unique(y))
        #check stopping criteria
        if(depth >= self.max_depth | number_of_labels == 1 | number_of_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        #find best split
        feat_idxs = np.random.choice(number_of_features, self.random_state, replace=False)
        best_feature,best_treshhold = self._best_split(X, y, feat_idxs)
        #create children nodes
        """left_idxs = X[:,best_feature] <= best_treshhold
        right_idxs = X[:,best_feature] > best_treshhold"""

        left_idxs, right_idxs = self._split(X[:,best_feature], best_treshhold)

        #recursively call the function

        left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)

        return Node(best_feature, best_treshhold, left, right)


    def fit(self, X, y):
        #tbp
        #döntési fa tanítása
        self.root = self._grow_tree(X, y)
        return None
    
    def _traverse_tree(self, x, node):
        #döntési fa bejárása
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def predict(self, X):
        #tbp
        #regresszios ertek meghatarozasa
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def print():
        #döntési fa ábrázolása
        #tbp
        return None

In [23]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

#Minta adatforrás
wine = load_wine()
X = wine.data # adatok
y = wine.target # címkék/célváltozók

#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő

#Döntési fa tanítása
dt = DecisionTree()
dt.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = dt.predict(X_test)
print(y_pred)

dt.print()

TypeError: '<=' not supported between instances of 'float' and 'NoneType'